In [81]:
!! pip install matplotlib
!! pip install -U scikit-learn

['Requirement already satisfied: scikit-learn in /Users/ysrsnmz/opt/anaconda3/lib/python3.9/site-packages (1.1.1)',
 'Requirement already satisfied: threadpoolctl>=2.0.0 in /Users/ysrsnmz/opt/anaconda3/lib/python3.9/site-packages (from scikit-learn) (2.2.0)',
 'Requirement already satisfied: joblib>=1.0.0 in /Users/ysrsnmz/opt/anaconda3/lib/python3.9/site-packages (from scikit-learn) (1.1.0)',
 'Requirement already satisfied: numpy>=1.17.3 in /Users/ysrsnmz/opt/anaconda3/lib/python3.9/site-packages (from scikit-learn) (1.20.3)',
 'Requirement already satisfied: scipy>=1.3.2 in /Users/ysrsnmz/opt/anaconda3/lib/python3.9/site-packages (from scikit-learn) (1.7.1)']

In [82]:
import pylab
import numpy as np
import pandas as pd
import sklearn.feature_extraction
import json
import os
import copy
import seaborn as sns
import matplotlib.pyplot as plt

In [83]:
# Set default pylab stuff
pylab.rcParams['figure.figsize'] = (14.0, 5.0)
pylab.rcParams['axes.grid'] = True

In [84]:
# Version 0.12.0 of Pandas has a DeprecationWarning about Height blah that I'm ignoring
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [85]:
# This is the Alexa 100k domain list, we're not using the 1 Million just for speed reasons. Results
# for the Alexa 1M are given at the bottom of the notebook.
alexa_dataframe = pd.read_csv('data/alexa_100k.csv', names=['rank','uri'], header=None, encoding='utf-8')
alexa_dataframe.head(20)

,rank,uri
0,1,facebook.com
1,2,google.com
2,3,youtube.com
3,4,yahoo.com
4,5,baidu.com
5,6,wikipedia.org
6,7,amazon.com
7,8,live.com
8,9,qq.com
9,10,taobao.com


In [86]:
# Okay for this exercise we need the 2LD and nothing else
import tldextract

def domain_extract(uri):
    ext = tldextract.extract(uri)
    if (not ext.suffix):
        return np.nan
    else:
        return ext.domain

alexa_dataframe['domain'] = [ domain_extract(uri) for uri in alexa_dataframe['uri']]
del alexa_dataframe['rank']
del alexa_dataframe['uri']
alexa_dataframe.head()

,domain
0,facebook
1,google
2,youtube
3,yahoo
4,baidu


In [87]:
alexa_dataframe.tail()

,domain
99995,rhbabyandchild
99996,rm
99997,sat1
99998,nahimunkar
99999,musi


In [88]:
# It's possible we have NaNs from blanklines or whatever
alexa_dataframe = alexa_dataframe.dropna()
alexa_dataframe = alexa_dataframe.drop_duplicates()

# Set the class
alexa_dataframe['class'] = 'legit'

# Shuffle the data (important for training/testing)
alexa_dataframe = alexa_dataframe.reindex(np.random.permutation(alexa_dataframe.index))
alexa_total = alexa_dataframe.shape[0]
print ('Total Alexa domains %d' % alexa_total)

Total Alexa domains 91369


In [89]:
alexa_dataframe.head()

,domain,class
13310,flonga,legit
61979,simplepressrelease,legit
51885,watchdub,legit
74223,punediary,legit
33995,oceanup,legit


In [90]:
# Read in the DGA domains
dga_dataframe = pd.read_csv('data/dga_domains.txt', names=['raw_domain'], header=None, encoding='utf-8')

# We noticed that the blacklist values just differ by captilization or .com/.org/.info
dga_dataframe['domain'] = dga_dataframe.applymap(lambda x: x.split('.')[0].strip().lower())
del dga_dataframe['raw_domain']

# It's possible we have NaNs from blanklines or whatever
dga_dataframe = dga_dataframe.dropna()
dga_dataframe = dga_dataframe.drop_duplicates()
dga_total = dga_dataframe.shape[0]

# Set the class
dga_dataframe['class'] = 'dga'

print ('Total DGA domains %d' % dga_total)

Total DGA domains 2664


In [91]:
dga_dataframe.tail()

,domain,class
2664,xdbrbsbm,dga
2665,xdfjryydcfwvkvui,dga
2666,xdjlvcgw,dga
2667,xdrmjeu,dga
2668,xflrjyyjswoatsoq,dga


In [94]:
# Concatenate the domains in a big pile!
all_domains = pd.concat([alexa_dataframe, dga_dataframe], ignore_index=True)
del all_domains['class']
print(all_domains)

                   domain
0                  flonga
1      simplepressrelease
2                watchdub
3               punediary
4                 oceanup
...                   ...
94028            xdbrbsbm
94029    xdfjryydcfwvkvui
94030            xdjlvcgw
94031             xdrmjeu
94032    xflrjyyjswoatsoq

[94033 rows x 1 columns]


In [95]:
#!/usr/bin/python

import math
import pickle

accepted_chars = 'abcdefghijklmnopqrstuvwxyz '

pos = dict([(char, idx) for idx, char in enumerate(accepted_chars)])

def normalize(line):
    """ Return only the subset of chars from accepted_chars.
    This helps keep the  model relatively small by ignoring punctuation, 
    infrequenty symbols, etc. """
    return [c.lower() for c in line if c.lower() in accepted_chars]

def ngram(n, l):
    """ Return all n grams from l after normalizing """
    filtered = normalize(l)
    for start in range(0, len(filtered) - n + 1):
        yield ''.join(filtered[start:start + n])

def train():
    """ Write a simple model as a pickle file """
    k = len(accepted_chars)
    # Assume we have seen 10 of each character pair.  This acts as a kind of
    # prior or smoothing factor.  This way, if we see a character transition
    # live that we've never observed in the past, we won't assume the entire
    # string has 0 probability.
    counts = [[10 for i in range(k)] for i in range(k)]

    # Count transitions from big text file, taken 
    # from http://norvig.com/spell-correct.html
    for line in open('big.txt'):
        for a, b in ngram(2, line):
            counts[pos[a]][pos[b]] += 1

    # Normalize the counts so that they become log probabilities.  
    # We use log probabilities rather than straight probabilities to avoid
    # numeric underflow issues with long texts.
    # This contains a justification:
    # http://squarecog.wordpress.com/2009/01/10/dealing-with-underflow-in-joint-probability-calculations/
    for i, row in enumerate(counts):
        s = float(sum(row))
        for j in range(len(row)):
            row[j] = math.log(row[j] / s)

    # Find the probability of generating a few arbitrarily choosen good and
    # bad phrases.
    good_probs = [avg_transition_prob(l, counts) for l in open('good.txt')]
    bad_probs = [avg_transition_prob(l, counts) for l in open('bad.txt')]

    # Assert that we actually are capable of detecting the junk.
    assert min(good_probs) > max(bad_probs)

    # And pick a threshold halfway between the worst good and best bad inputs.
    thresh = (min(good_probs) + max(bad_probs)) / 2
    pickle.dump({'mat': counts, 'thresh': thresh}, open('gib_model.pki', 'wb'))

def avg_transition_prob(l, log_prob_mat):
    """ Return the average transition prob from l through log_prob_mat. """
    log_prob = 0.0
    transition_ct = 0
    for a, b in ngram(2, l):
        log_prob += log_prob_mat[pos[a]][pos[b]]
        transition_ct += 1
    # The exponentiation translates from log probs to probs.
    return math.exp(log_prob / (transition_ct or 1))

if __name__ == '__main__':
    train()

In [97]:
#!/usr/bin/python

import pickle
import gib_detect_train

model_data = pickle.load(open('gib_model.pki', 'rb'))


while True:
    for x in all_domains.values:
        for y in x:
            model_mat = model_data['mat']
            threshold = model_data['thresh']
            print (y, gib_detect_train.avg_transition_prob(y, model_mat) > threshold)
    break
    
    

flonga True
simplepressrelease True
watchdub True
punediary True
oceanup True
specialdatabases True
trikky False
paksoldiers True
zzn False
eureka-startups True
rba False
realage True
programmersheaven True
7m True
525happy True
akibahobby True
bundesliga-streams True
ping2it True
wallpaper23 True
comze True
oemresource True
portaportal True
freedl True
eme2000 True
gay-jp False
zapworld True
almubasher True
astrosfera True
8799 True
garnethill True
alterclicks True
pvponline False
dailymarkets True
asiaflash True
lasd False
blog1 True
todoar True
ab-in-den-urlaub-deals True
bizchair True
moninet True
sakarya True
free2doo True
1089u True
ezibuy False
gracobaby True
raovatonline True
marodi True
insightbb True
metrobot True
ing2ing True
testosterona True
raven-seo-tools True
cameroonvoice True
yqmr False
lindenstrasse True
neopets True
hughesnet True
weatheronline True
ifly True
tyc False
arkivmusic True
podberi-trenajor True
spotify True
fulltiltpoker True
free-review True
outdooracti

welho True
cakgames True
qualipub True
dvd False
thehunt True
gufile True
automation True
actitudfem True
minne True
zhanbu True
avclabs False
justsweep True
buaa False
musyuusei-douga True
hisense True
binck True
matematyka True
myjobmag False
grumpycats True
policja True
bitparking True
desktopmania True
iapp True
dragonsmobile True
psasurveys True
trendyday True
sklepy24 True
thebox True
longurl True
lavoixdunord True
medina-fm True
letsunlockiphone True
ypfs False
linkshares True
99acres True
numbersusa True
chinazhw True
wsodownloads True
tech True
meimei0 True
thecatsite True
vingrad True
colorzilla True
bokus True
fatquartershop True
zooporntube True
glitter-graphics True
ashorooq True
coair True
connexus True
codehaus True
eroanimedouga True
tarafdari True
lifestylewizard True
powerschool True
javdict True
cowful True
camiseteria True
autokrata True
resiusa True
skladchik True
hurley True
srclick False
fg-site True
trjcn False
halfpasthuman True
freeurlwebsite True
hyipman True

vbinspired True
rockettheme True
rayhanzhampiet True
asrilanka True
theclickcheck True
ljlzx False
sarkari-naukri True
ciudadseva True
lbl False
actstudent True
networkcomputing True
dfcufinancial True
industrialthemes True
imvu False
libimseti True
whogohost True
phpbb-seo False
blogsynergy True
sdrs False
ispeak True
webkul False
haon True
mamashai True
pvc123 False
internationalcupid True
paokmania True
scczx False
machinerytrader True
ricercadiricette True
bitgravity True
farnet True
14099 True
mediatico True
hdletv False
rakumachi True
directoryaddsite True
onedayonly True
qianhuaweb False
pasionliberal True
i8866 True
cite True
fortrader True
gigapan True
progress-money True
realitysandwich True
catavencii True
rslinks True
ticketleap True
admeld True
buyanyseat True
popeyes True
mysupercheap True
sweetamoris True
blog-raskruti True
netcars True
ecb False
trtsp True
journalhome True
theahl True
discountasp True
hillnews True
icondock True
koifaire True
getoffmyinternets True
shan

cheaperseeker True
kenkoucorp True
turktime True
secure-signup True
bbcamericashop True
dlbazar False
spinbot True
alexaboostup True
nettiauto True
mydoc False
thepaintmovement True
diamondbank True
lotteimall True
blogbugs False
loadimpact True
idols69 True
dtinsure True
ebayrtm False
dizzedlink True
webdesignerdepot True
dduu False
uschoolnet True
onsen True
efshop False
rbfonline True
clickvoyager True
raunchygfs False
3suisses True
hthayat True
fooddemocracynow True
autopro True
depkeu False
gillyhicks True
donbaler True
6m True
weburlsearch True
karaoke-version True
songswale True
difi True
inexpenshop True
city-link True
connectify True
bangren168 True
dogoo True
dewa True
sfedu True
binushacker True
izlep True
youpouch True
bankwest True
directoryfuse True
bkns False
blowjobvision True
gardensalive True
downws False
homefuckk True
bfi0 False
superbiiz True
chupamobile True
ln-n-tax False
ons True
diskusiwebhosting True
gotujmy False
qroo False
111job True
zagolovki True
dutchlea

thetechgame True
gamesfreak True
onamae True
telaerotica True
webforge True
opentraders True
vistek True
gamma-ic True
adobeconnect True
enjoycookinggames True
binvesto True
elfinanciero True
trendhunter True
war2glory True
askmonty True
bundesanzeiger True
inboundmarketing True
preissuchmaschine True
saiter True
tkne False
eve-ru True
adhitprofits True
dudesnude True
ferronetwork True
allaturkaa True
bits-pilani True
boardhost True
poetryfoundation True
elitefox True
tutorialchip True
spankingtube True
escortforum True
womenshealthreport True
geomix True
exteen True
musicaspraouvir True
btopenworld True
bitwn True
nazcea False
magentic True
24freeads True
28hse False
farskids328 True
05wan True
worldometers True
royalptr True
persian-music True
roadcontrol True
peliculas21 True
opentracker True
tubezaur True
sportlemon True
pkpolitics False
bricozone True
thedirectorycheap True
gaytube1 True
seoflesh True
emart True
antoniocs True
diziler True
24horas True
kugo True
bestmoviesevernews

...

casadasgostosas True
linkhay True
utahrealestate True
psybernet True
pcsbanking True
money-box True
web-stat True
devocionario True
iheartpublix False
covertvideopress True
forumjuridico True
100order True
canoe True
dmacc True
kwtbox False
fetishshrine True
legoujie False
articlesbd True
rwth-aachen True
gametama True
xnews False
autocatch True
highlandparkinternetdesigns True
wl False
uibia True
caijj False
pr0gramm True
sportsinjuryclinic True
gorz False
levashov True
tra3a False
sverigesradio True
backlinkwatch True
templatesbox True
kimono True
1001pelit True
elhasad True
freakz True
videodirectory8 True
autogeek True
alternativeto True
virpus True
lansik True
gerasanews True
fwisp True
moto-station True
211brand True
appointment-plus True
webdirectorieslist True
weeklystandard True
hqxnxx False
travelservicebangkok True
paid2clik True
autoinsurancetips True
myrightad True
sexstoriespost True
wcxiaoshuo False
cdrking True
kodyrabatowe True
jinjitter True
ctfeshop True
cnops False


cucsnva67d20f52esjuasf12hsoyoxirf52lt False
cuijmuljysivscwe False
cumtc49a67izivhsjwn20evbrcsdzo51bymy False
cunirgugp True
cuvbenzf False
cuveztrnrgnshbgp False
cva57kwavhvb28gqe51j36iwjqhvbzi25p12ku False
cvavgygvk17ove31ero41gqltoyhrgyfzes False
cvcejggrs False
cvcndfdb False
cvdw100 False
cvmxadoqr False
cwaxcto11bxpwatb68f52o11iuotoqcxlxl68 False
cwcbfoqyo False
cwgzo51gtcxe11m39pqltk47fwe41f22b68aydz False
cwj16b18a57dyc19gsaspwn20p52mqn40mql18dq False
cwqhrwknk False
cwryzguoqnj False
cxbjojg False
cxgdoyppljy False
cxgfaham False
cybxcikisigkmqtl False
cyd30hsm69lufwiwg63fqdqjqc39jritkue51 False
cygitzxcva False
cyh14fqh14h54e41bukveqa37j66aql18i35g53i15 False
cyh24lzi55a67prhte61f62btotiulxlri55pw False
cyosongjihugkjbg False
cyvsemhrwterkxgqoinmvyeu False
cyyxghmbwgcaovtcajrtcnslz False
czaqo31j46otc19e61gtluawfydvfyj16gzjw False
czcvn10hqesn10lucvbzbqawlsiynra37n60 False
cznvg43hxlqduh14gqmunwmrc29b18b18fyc19 False
czonpjw False
czozexcp False
d02880bcae False
d20e31g33g53e

hgncyvomcjg False
hgptprcqns False
hhcsdgadid False
hhfqupqvzk False
hhziyzyy False
hicqgipogsjulrgn False
hidifzbettjuadfh False
hidujyjab False
hieeowin True
hievgwnfzdi False
hifivwxxwaemsopofljtgprlugyp False
himoviwam True
hinqbcy False
hiplksflttfkpsxn False
hiqwovum False
hiqyhyxbj False
hiuknnpfu False
hjkmubhjfw False
hjqaurzy False
hjspnsbj False
hjssatsn False
hjsvxaunpsn False
hkeurwclmqpfcytll False
hlfjctup False
hlhewvzso False
hlmxitb False
hlojmyew False
hltsuirx False
hmzlzpfrvmp False
hnckqpea False
hniitysuwprckvzs False
hnkhhfek False
hnrukcf False
hntbr11 False
hntbw00 False
hntbw06 False
hntbw13 False
hntbw52 False
hntbw55 False
hntbw918 False
hnyefrzh False
hofkvirdypnfeyhmngyldmygsc False
honurkip True
houw100 False
hprndfmackg False
hqcgojuj False
hqcivfc False
hqcwtvfxlikvhh False
hqpplnyc False
hqtizayjkk False
hquoamodmeqvpjesogubystfi False
hrhjylywixq False
hrkusbnevtmyisab False
hrqnkqld False
hrrnixjozgx False
hruizxst False
hrusnwzn False
hsaznzgqaqo61

nwdvufzkpszkvxxk False
nwfqmbk False
nwm29dsdtd30l28h14o21iud10g43prnvfvoyhs False
nwp62fwkwdwd60pvb48ftasgrmqhtkqpzl48 False
nwpwdhet False
nwpykqeizraqthry False
nwzfmtpemzh False
nxugmsv False
nxzlwtfmuoz False
nyeifzj False
nyfzggqks False
nyjuivp12n30nqougvn30kxkrdrn30hwmqjs False
nyjyqxitn False
nykrbsk False
nyow100 True
nyrhkmuk False
nyzpnugwn False
nzdcbhraa False
nzdtjwblgau False
nzkkxijmgl False
nzmnmfitrm False
nzphegugexhilbvwqswogwodlf False
nzpwnpuw False
nzyywecnn False
o21j36gxotcxesmvawnvjtlxexn50a27h44lx False
o31czgqprj36n30o31l18myg53fxb58pvi45k67js False
o31muexdvo11b68f42m39pub58krnsg13atiwox False
o41ewmycwdugqn30ltgxe21n50iun10itfuez False
o51ezc19ascto61p12l68b48nwl18etm29etjqdv False
o51hvf12j16bxgwkzpuhudxe21esk37lzg23jr False
o61p62h34h44frmvo11jydvdyh64dsoqcxc49j66 False
o61pxn40bwfwcqi25k37k37cxn40mqeqi15m49h14 False
oakw100 False
oanbnuemftr False
oaxijyzx False
obathbxgyuw False
obgrcxuqunmquthx False
objlhoht False
obllgktbp False
obtrslbct False
ocf

sotkyzhortindmpeyamtcivnvy False
spdenojggmdrlixc False
spoucknzn True
spphczekzysdypqb False
spqmnhanu False
sqgninvq False
sqjzieauibc False
sqpsxuggvou False
squrubzzdm False
sqwqbayzgl False
srodiydkjgamjskltifbbqpfqkfm False
sronmbfao False
srotcxurd False
steqvhuhrqsmynoh False
stgherp True
stlw100 False
strkzmcq False
stwzzhgy False
suhumoqdo False
suvnelk False
suxlukeatupw False
svjmxvn False
svknyhls False
swbmlmrvtv False
swdpumrxz False
swgcotcwn False
swjhjugnk False
sxayewpi False
sxcvxec False
sxhumdgvv False
sxkiifqgzmsjvxzn False
sxmcvcyt False
syhpoyykyz False
syjqyvrpyohlexgj False
synlfum False
sywddeixwjp False
sywleisrsstsqoic False
szazugop False
szgewrf False
talw100 True
tapojlzv False
tbhsasmkp False
tbpqsyobc False
tbzduewj False
tcbubesgbupfzpztqotdamlceu False
tcgfmvaq False
tcpgbtgvt False
tcyknaetztukrdeyprcyz False
tdahfyfqzhzhvkmjschyfaworgxg False
tdlbnxq False
tdlxjkhqvw False
tdndpphrtyniynvz False
teeaypxcbff False
tefoyrbhg False
tehjvmj False
teiw